In [1]:
#
# Copyright (C) 2024 by Sonja Filiposka <sonja.filiposka@finki.ukim.mk>
#
# This code is licensed under a Creative Commons Attribution 4.0 International License. (see LICENSE.txt for details)
#
# [TODO]General Description - this notebook is used to extract the delay and the handover information from OMNET output vector file
# It creates two types of output files: 
#    - mobile network communication delay as reported by OMNET
#      - a csv file with the delay for each communication exchange
#    - initial positioning and migration files that are used as input for CloudSim 
#       - initial positioning file defines the start and end time for each car/service and the initial community 
#         based on location of tha car and the location of the base stations (see comments in code)
#       - migration file defines the time stamp when a car moves from one community to another based on relative location to the nearest base station
#

In [3]:
def dataset_omnet(vector_omnet, output):

    # parameters
    # 1 = $vector_omnet - omnet input.file name
    # 2 = $output       - output.file name
        
    # First prepare the output file from OMNET, vector-0.vec to suit version 2.0
    !split -d -b 500m $vector_omnet
    !sed -i 's/version 3/version 2/' x00
    !cat x* > vector-1.vec
    # clean
    !rm x*
    # requires to have netperfmeter installed: sudo apt-get install netperfmeter
    !extractvectors vector-1.vec results.bz2 "distance" \
        "measuredSinrDl" "measuredSinrUl" "rcvdSinrDl" "averageCqiDl" \
        "servingCell" "rlcDelayDl" "rlcPacketLossTotal" "rlcPduDelayDl" \
        "rlcPduPacketLossDl" "rlcPduThroughputDl" "rlcThroughputDl"\
        "receivedPacketFromLowerLayer" 

    # get data file: results (it deletes results.bz2)
    if os.path.isfile('results'):
        !rm results
    !bzip2 -d results.bz2
    
    # removing unnecessary information
    !grep 'car' results > r_cars
    !sed -i -e 's/NRSeveralBSALC.//' -e 's/:vector//'  -e 's/ ETV//' \
        -e 's/(packetBytes)//' r_cars
    
    # clean
    !rm vector-1.vec 
    
    cols=['Time','Event','Object','Vector','Split','Value']
    data = vx.read_csv("r_cars", sep='\t', header=None,
        #names=['#','Time','Event','Object','Vector','Split','Value'],
        names=['#','Time','Event','Object','Vector','Value'],
        usecols=['Time','Object','Vector','Value'],
        convert=True, chunk_size=150_000_000) 
    print(data)

    #data = data['#'].drop
    data.Object = data.Object.str.replace('"', '')
    data.Vector = data.Vector.str.replace('"', '')    

    data.export_csv(output, index=False, sep='\t')
    
    # clean
    !rm results r_cars* 
    del data
    
    return 1 


In [ ]:
# measuring execution time
%load_ext autotime

# extract the delay and the handover information from omnet output vector file
maxTime = 1800
communities = 9

#parametrised calls of notebook
import numpy as np
import pandas as pd
import vaex as vx
import os.path

path="/home/jupyter/notebook/OMNET6.0/"
sumo_files = "dataset_AI_input/fdc_signals_"
omnet_files = "VoipDl-Urban-xxxx/VoipDl-Urban-"
#cars = np.array([4928, 4951, 4955, 5712, 5734, 5749, 6900, 6908, 6923, 8589, 8619, 8620])
cars = np.array([8589, 8619, 8620])
out = "dataset_AI_output/"
outFile = "_AI.csv"
# initialPositioning-xxxx.txt and migrations-xxxx.txt will also be.ipynb_checkpoints/created

for i in cars:
    # create omnet dataframe
    v_omnet = path + omnet_files + str(i) + "/vector-0.vec"
    output = path + out + str(i) + "_omnet" + outFile
    print("STEP 1) omnet input : ", v_omnet)
    print("STEP 1) omnet output : ", output)
    # extract dataset from OMNET vector
    omnet_exit_code1 = dataset_omnet(v_omnet, output)
    print(omnet_exit_code1) 

STEP 2) sumo input :  dataset_AI_output/8589_sumo_AI.csv
STEP 2) sumo output :  dataset_AI_output/8589_sumo_2_AI.csv


/tmp/ipykernel_5809/1462651122.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pd = pd.read_csv(s_input, sep='\t', header=None,


(20679146, 10)
0           veh_id
1                0
2                0
3                0
4                0
             ...  
20679141       888
20679142       962
20679143       972
20679144       975
20679145       999
Name: veh_id, Length: 20679146, dtype: object
      t veh_id          x          y  angle speed   pos        lane slope  \
1   0.0      0  -0.485544  38.341737  39.92  0.00  5.10  33199229_0  0.00   
2  0.01      0  -0.485544  38.341737  39.92  0.03  5.10  33199229_0  0.00   
3  0.02      0  -0.485544  38.341737  39.92  0.05  5.10  33199229_0  0.00   
4  0.03      0  -0.485544  38.341737  39.92  0.08  5.10  33199229_0  0.00   
5  0.04      0  -0.485544  38.341737  39.92  0.10  5.10  33199229_0  0.00   

  signals  
1       0  
2       8  
3       8  
4       8  
5       0  
(20679145, 24) 